In [1]:
from operator import itemgetter
from langchain.memory import ConversationBufferMemory
from langchain_core.prompts import ChatPromptTemplate,MessagesPlaceholder
from langchain_core.runnables import RunnablePassthrough,RunnableLambda
from langchain_openai import ChatOpenAI

In [2]:
model = ChatOpenAI(model_name="gpt-4o-mini")
prompt = ChatPromptTemplate.from_messages(
    [
        ("system","You are a helpful chatbot"),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human","{input}")
    ]
)

In [3]:
memory = ConversationBufferMemory(return_messages= True,memory_key="chat_history")

C:\Users\Admin\AppData\Local\Temp\ipykernel_22852\3148526874.py:1: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(return_messages= True,memory_key="chat_history")


In [4]:
memory.load_memory_variables({})  # 메모리 변수를 빈 딕셔너리로 초기화합니다.


{'chat_history': []}

In [5]:
runnable = RunnablePassthrough.assign(
    chat_history=RunnableLambda(memory.load_memory_variables)
    | itemgetter("chat_history")  # memory_key 와 동일하게 입력합니다.
)


In [6]:
runnable.invoke({"input": "hi"})


{'input': 'hi', 'chat_history': []}

In [7]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful chatbot"),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{input}"),
    ]
)


In [8]:
runnable.invoke({"input": "hi!"})


{'input': 'hi!', 'chat_history': []}

In [9]:
chain = runnable | prompt | model


In [11]:
memory.load_memory_variables({})


{'chat_history': []}

In [12]:
# 입력된 데이터와 응답 내용을 메모리에 저장합니다.
memory.save_context(
    {"human": "만나서 반갑습니다. 제 이름은 테디입니다."}, {"ai": response.content}
)

# 저장된 대화기록을 출력합니다.
memory.load_memory_variables({})


{'chat_history': [HumanMessage(content='만나서 반갑습니다. 제 이름은 테디입니다.', additional_kwargs={}, response_metadata={}),
  AIMessage(content='안녕하세요, 테디님! 만나서 반갑습니다. 어떻게 도와드릴까요?', additional_kwargs={}, response_metadata={})]}

In [13]:
# 이름을 기억하고 있는지 추가 질의합니다.
response = chain.invoke({"input": "제 이름이 무엇이었는지 기억하세요?"})
# 답변을 출력합니다.
print(response.content)


네, 테디님이라고 하셨습니다. 다른 질문이나 이야기하고 싶은 주제가 있으신가요?


In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate,MessagesPlaceholder
from langchain.memory import ConversationBufferMemory,ConversationSummaryMemory
# 모델 초기화
llm = ChatOpenAI(model_name="gpt-4o-mini")
# 프롬프트 초기화
prompt =  ChatPromptTemplate.from_messages(
    ("system", "You are a helpful chatbot"),
    MessagesPlaceholder(variable_name="chat_history"),
    ("human", "{input}"),
)
# 메모리 초기화
memory = ConversationBufferMemory(return_messages=True,memory_key='chat_history')
# chain 만들기